In [1]:
import numpy as np
from PIL import Image
import ctypes
import matplotlib.pyplot as plt
import random

In [2]:
asia = Image.open('test_images/asie/asia.jpg')
greek = Image.open('test_images/greek/greek.jpg')
chateau = Image.open('test_images/chateau/train_chateau.jpg')
predict_asia = Image.open('test_images/asie/predict_asie.jpg')
predict_greek = Image.open('test_images/greek/predict_grec.jpg')
predict_chateau = Image.open('test_images/chateau/predict_chateau.jpg')

In [3]:
newsize = (50,50)

In [4]:
asia_new_size = asia.resize(newsize)
greek_new_size = greek.resize(newsize)
chateau_new_size = chateau.resize(newsize)
predict_asia_new_size = predict_asia.resize(newsize)
predict_greek_new_size = predict_greek.resize(newsize)
predict_chateau_new_size = predict_chateau.resize(newsize)

In [5]:
# asia_new_size.show()
# greek_new_size.show()
# chateau_new_size.show()

In [6]:
asia_data = np.asarray(asia_new_size)
greek_data = np.asarray(greek_new_size)
chateau_data = np.asarray(predict_chateau_new_size)
predict_asia_data = np.asarray(predict_asia_new_size)
predict_greek_data = np.asarray(predict_greek_new_size)
predict_chateau_data = np.asarray(predict_chateau_new_size)

X = np.array([asia_data, greek_data, chateau_data], dtype=ctypes.c_float)
X_test = np.array([predict_asia_data, predict_greek_data, predict_chateau_data], dtype=ctypes.c_float)
X_flat = X.flatten()
X_flat = X_flat/255 * 2 - 1
# w_array = w_array/255 * 2 - 1

X_test_flat = X_test.flatten()
X_test_flat = X_test_flat/255 * 2 - 1

In [7]:
print(len(X_flat))

22500


In [8]:
print(len(X))

3


In [9]:
Y = np.array([[1, -1, -1], [-1, 1, -1], [-1, -1, 1]], dtype=ctypes.c_float)
Y_flat = Y.flatten()
Y_test = np.array([[1, -1, -1], [-1, 1, -1], [-1, -1, 1]], dtype=ctypes.c_float)
Y_test_flat = Y_test.flatten()
print(Y_flat)
print(Y_test_flat)

[ 1. -1. -1. -1.  1. -1. -1. -1.  1.]
[ 1. -1. -1. -1.  1. -1. -1. -1.  1.]


In [10]:
my_lib = ctypes.CDLL(r"..\rust_lib\target\release\rust_lib.dll")

my_lib.delete_float_array.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
]

my_lib.generate_random_mpl_w.argtypes = [ctypes.POINTER(ctypes.c_float), ctypes.c_int32]
my_lib.generate_random_mpl_w.restype = ctypes.POINTER(ctypes.c_float)

my_lib.delete_float_array.restype = None

layers = np.array([len(X_flat)/len(X), 3], dtype=ctypes.c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
layers_pmc = (ctypes.c_float * len(layers))(*layers)
layers_ptr_pmc = ctypes.cast(layers_pmc, LP_c_float)

print(len(layers_pmc))
number_of_layers = len(layers_pmc)
print(layers)

2
[7.5e+03 3.0e+00]


In [11]:
# w_array_ptr = my_lib.generate_random_mpl_w(layers_ptr_pmc, number_of_layers)
# w_array = np.ctypeslib.as_array(w_array_ptr, ((7501*3,))) #3*16 + 17*16 + 17*3
#
# LP_c_float = ctypes.POINTER(ctypes.c_float)
# x_to_predict = (ctypes.c_float * len(X_flat))(*X_flat)
# x_to_predict_ptr = ctypes.cast(x_to_predict, LP_c_float)
# LP_c_float = ctypes.POINTER(ctypes.c_float)
# y_to_predict = (ctypes.c_float * len(Y_flat))(*Y_flat)
# y_to_predict_ptr = ctypes.cast(y_to_predict, LP_c_float)
# my_lib.multi_layer_perceptron_training.argtypes = [
#     ctypes.POINTER(ctypes.c_float),
#     ctypes.POINTER(ctypes.c_float),
#     ctypes.POINTER(ctypes.c_float),
#     ctypes.c_int32,
#     ctypes.c_int32,
#     ctypes.c_int32,
#     ctypes.c_float,
#     ctypes.c_int32,
#     ctypes.POINTER(ctypes.c_float),
#     ctypes.c_int32,
#     ctypes.c_bool,
# ]
# my_lib.multi_layer_perceptron_training.restype = ctypes.POINTER(ctypes.c_float)

In [12]:
w_array_ptr = my_lib.generate_random_mpl_w(layers_ptr_pmc, number_of_layers)
w_array = np.ctypeslib.as_array(w_array_ptr, ((7501*3,))) #3*16 + 17*16 + 17*3
print(w_array)
LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict = (ctypes.c_float * len(X_flat))(*X_flat)
x_to_predict_ptr = ctypes.cast(x_to_predict, LP_c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
y_to_predict = (ctypes.c_float * len(Y_flat))(*Y_flat)
y_to_predict_ptr = ctypes.cast(y_to_predict, LP_c_float)

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict_test = (ctypes.c_float * len(X_test_flat))(*X_test_flat)
x_to_predict_test_ptr = ctypes.cast(x_to_predict_test, LP_c_float)
LP_c_float = ctypes.POINTER(ctypes.c_float)
y_to_predict_test = (ctypes.c_float * len(Y_test_flat))(*Y_test_flat)
y_to_predict_test_ptr = ctypes.cast(y_to_predict_test, LP_c_float)

my_lib.multi_layer_perceptron_accuracy.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_float,
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_bool,
]
my_lib.multi_layer_perceptron_accuracy.restype = ctypes.POINTER(ctypes.c_float)

[ 9.18334081e-07  1.10262417e-05  9.47610260e-06 ... -1.95219036e-06
 -1.27050625e-05  5.17146054e-06]


In [13]:
print(max(w_array))
print(min(w_array))
print(sum(w_array)/len(w_array))

1.3328218e-05
-1.332922e-05
3.1476469394680465e-08


In [14]:
pmc_trained_model_accuracy_ptr = my_lib.multi_layer_perceptron_accuracy(
    w_array_ptr,
    y_to_predict_ptr,
    x_to_predict_ptr,
    len(X),  # number of inputs
    y_to_predict_test_ptr,
    x_to_predict_test_ptr,
    len(X_test),
    int(len(X_flat)/len(X)),  # dimension of inputs
    3,  # number of classes to predict
    0.0001,  # learning rate
    10000,  # epochs
    1,
    layers_ptr_pmc,  # pointeur vers la forme de pmc
    number_of_layers,  # number of layers
    True,
)

pmc_trained_model = np.ctypeslib.as_array(pmc_trained_model_accuracy_ptr, (7501*3,))

In [15]:
print(len(X))

3


In [31]:
# pmc_trained_model_ptr = my_lib.multi_layer_perceptron_training(
#     w_array_ptr,
#     y_to_predict_ptr,
#     x_to_predict_ptr,
#     len(X),  # number of inputs
#     int(len(X_flat)/len(X)),  # dimension of inputs
#     3,  # number of classes to predict
#     0.5,  # learning rate
#     10,  # epochs
#     layers_ptr_pmc,  # pointeur vers la forme de pmc
#     number_of_layers,  # number of layers
#     True,
# )
#
# pmc_trained_model = np.ctypeslib.as_array(pmc_trained_model_ptr, (7501*3,))
# #print(pmc_trained_model)


ArgumentError: argument 7: TypeError: Don't know how to convert parameter 7

In [16]:
# asia_data = np.asarray(asia_new_size)
# greek_data = np.asarray(greek_new_size)
# south_america_data = np.asarray(south_america_new_size)
image_a_predire_data = np.array([predict_asia_data, predict_chateau_data, predict_greek_data],dtype=ctypes.c_float)
image_a_predire_data_flat=image_a_predire_data.flatten()
# new_X = np.array([south_america_data, asia_data, greek_data], dtype=ctypes.c_float)
# new_flat_X = X.flatten()

In [17]:
print(len(image_a_predire_data))
print(len(image_a_predire_data_flat))

3
22500


In [18]:
my_lib.multi_layer_perceptron_predict.argtypes = [
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.c_int32,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int32,
    ctypes.c_bool,
]

my_lib.multi_layer_perceptron_predict.restype = ctypes.POINTER(ctypes.c_float)

LP_c_float = ctypes.POINTER(ctypes.c_float)
x_to_predict_mlp = (ctypes.c_float * len(image_a_predire_data_flat))(*image_a_predire_data_flat)
x_to_predict_ptr_mlp = ctypes.cast(x_to_predict_mlp, LP_c_float)

pmc_predict_ptr = my_lib.multi_layer_perceptron_predict(
    pmc_trained_model_accuracy_ptr,  # pointeur vers le modele entrainé
    x_to_predict_ptr_mlp,  # pointeur vers le dataset à prédire
    len(image_a_predire_data),  # nombre d'éléments du dataset
    int(len(image_a_predire_data_flat)/len(image_a_predire_data)),  # dimension des éléments du dataset
    3,  # number of classes to predict
    layers_ptr_pmc,  # pointeur vers la forme de pmc
    number_of_layers,  # number of layers
    True,  # is classification
)

pmc_predict_model = np.ctypeslib.as_array(pmc_predict_ptr, (3 * len(image_a_predire_data),))
# print(pmc_predict_model)
# print(len(X_to_predict))
# print(len(X_to_predict_flat))


NameError: name 'pmc_trained_model_ptr' is not defined

In [19]:
for j in range(0,len(pmc_predict_model)-2,3) :
    if pmc_predict_model[j] > pmc_predict_model[j+1] and pmc_predict_model[j] > pmc_predict_model[j+2] :
        print("L'image a deviner vient d'asie")
    elif pmc_predict_model[j+1] > pmc_predict_model[j] and pmc_predict_model[j+1] > pmc_predict_model[j+2] :
        print("L'image a deviner vient de grece")
    elif pmc_predict_model[j+2] > pmc_predict_model[j] and pmc_predict_model[j+2] > pmc_predict_model[j+1]:
        print("L'image a deviner vient d'amerique du sud")
    else :
        print('résultats non concluants')

NameError: name 'pmc_predict_model' is not defined

In [23]:
print(pmc_predict_model)

[-0.9992589   0.9996983   0.93401915 -0.99956805  0.9992659   0.8752813
 -0.9994039   0.99961466  0.8829671 ]
